# Regression Analyses (work in progress--more code and much more documentation to come)

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
e = create_engine('sqlite:///../Appendix/nvcu_db.db')
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import plotly.express as px

import sys
sys.path.insert(1, '../Appendix')
from helper_funcs import config_notebook, wadi
display_type = config_notebook(display_max_columns = 7,
                              display_max_rows = 8)
df_survey_results = pd.read_csv(
    '../Data_Prep/2023_survey_results.csv')


In [ ]:
# Creating a table of fall and spring bookstore sale totals for each 
# student:
# This will be based on the student demographics table, as 
# certain demographic items will play a role in students' spring 
# purchase totals.
df_sales = pd.read_sql(
    "select student_id, gender, college, level from \
curr_enrollment", con = e)
enrollment = len(df_sales)
df_sales

## Creating a new bookstore sales table that will ultimately get incorporated into the Appendix:

In [ ]:
rng = np.random.default_rng(seed = 225403)
df_sales['Fall'] = rng.normal(
    loc = 80, scale = 25, size = len(df_sales))

# spring_change = 3 + (rng.random(size = 1000)) * -6
spring_change = rng.normal(loc = 11, scale = 25, size = len(df_sales))
df_sales['Spring'] = df_sales['Fall'] + spring_change
# Modifying Spring totals based on demographic components:
spring_col = df_sales.columns.get_loc('Spring')
for i in range(len(df_sales)):
    # Unhealthy snacks were removed from the checkout aisle; 
    # this ended up reducing revenue among freshmen and sophomore
    # (who particularly liked these snacks)
    if df_sales.iloc[i]['level'] in ['Fr', 'So']:
        df_sales.iloc[i, spring_col] = (
            df_sales.iloc[i, spring_col] + rng.normal(
                loc = -21, scale = 3))
    # An intensive marketing campaign was carried out at STM and STL;
    # if it ended up being successful, it would then be implemented
    # at the other colleges also.
    if df_sales.iloc[i]['college'] in ['STM', 'STL']:
        df_sales.iloc[i, spring_col] = (
            df_sales.iloc[i, spring_col] + rng.normal(
                loc = 9, scale = 3))

# I'll leave in negative Fall and Spring values, as they could be 
# explained by refunds.

df_sales['Fall'] = df_sales['Fall'].round(2)
df_sales['Spring'] = df_sales['Spring'].round(2)
df_sales['Fall_Spring_Change'] = df_sales['Spring'] - df_sales['Fall']

df_sales

## Survey result analyses:

(Note: These analyses failed at least one linear regression suitability check--so I don't want to make them the main focus of this section.)

In [ ]:
px.scatter(df_sales, x = 'Fall', y = 'Spring')

In [ ]:
df_sales_pivot = df_sales.pivot_table(
    index = ['college', 'level'], values = 'Fall_Spring_Change', 
    aggfunc = 'mean').reset_index()
df_sales_pivot

In [ ]:
px.bar(df_sales_pivot, x = 'college', y = 'Fall_Spring_Change',
      color = 'level', barmode = 'group', text_auto = '.1f')

In [ ]:
sales_lr_1 = smf.ols(formula = "Spring ~ Fall + college + level + gender", 
                   data = df_sales) 
sales_lr_results_1 = sales_lr_1.fit()
sales_lr_results_1.summary()

## Checking the normality of our residuals

For more information on diagnostic tests within Statsmodels, see
https://www.statsmodels.org/dev/examples/notebooks/generated/regression_diagnostics.html
and https://www.statsmodels.org/stable/diagnostic.html .

The model-checking section of Learning Statistics with Python's Linear Regression chapter, available at https://ethanweed.github.io/pythonbook/05.04-regression.html#model-checking, is also a valuable reference. (It does contain some mild language.)

In [ ]:
px.histogram(sales_lr_results_1.resid)

In [ ]:
sms.omni_normtest(lr_results_1.resid)

In [ ]:
df_demographics = pd.read_sql(
    "select student_id, gender, college, level from \
curr_enrollment", con = e)
df_demographics

In [ ]:
df_survey_results = pd.read_sql("select * from survey_results", 
                                con = e)
df_survey_results.query("score < 100", inplace = True) 
# # Survey scores cannot
# be above 100; thus, for very high fall scores, the average fall-spring
# growth got constrained (as scores had less room to climb than did 
# lower scores). In order to prevent this phenomenon from influencing
# our regression, I decided to filter higher fall scores (which were
# more likely to result in spring scores of 100) out of our dataset.
# There are undoubtedly better ways of handling this ceiling effect
# for real-world applications, but this approach will suffice for
# a simulated example like this one.


df_survey_results

In [ ]:
px.histogram(df_survey_results, x = 'score')

In [ ]:
df_results_and_demographics = df_survey_results.merge(
    df_demographics, on = 'student_id', how = 'inner')

df_results_and_demographics

Creating a pivot table for our regression:

In order to determine the impact of college and level on students' fall and spring scores, we'll need to place those scores side by side. We can accomplish this by calling `df.pivot()` and passing 'season' to its `columns` argument.

In [ ]:
df_regression = df_results_and_demographics.pivot(
    index = ['student_id', 'gender', 'college', 'level'],
                        columns = 'season', 
                        values = 'score').reset_index().dropna()

df_regression['const'] = 1 # This value will prove useful
# within regression analyses that don't use Statsmodels' formula api 
# (which adds the constant on its own; 
# see https://www.statsmodels.org/stable/examples/notebooks/generated/formulas.html).
# Statsmodels does have an 
# add_constant() function that can accomplish this step for us,
# but it's even simpler to just add it within Pandas.
df_regression

In [ ]:
px.histogram(df_regression, x = ['Spring', 'Fall'], barmode = 'group')
# This code was based on the histogram documentation at:
# https://plotly.com/python/histograms/

In [ ]:
fig_fall_spring_comparison = px.scatter(
    df_regression, x = 'Fall', y = 'Spring',
title = 'Spring scores as a function of Fall scores')
fig_fall_spring_comparison

The following cell defines a function that, given a list of index values, can create a pivot table showing changes in fall/spring scores for each combination of those values. It is designed to work specifically with `df_regression` but could be modified to accommodate other DataFrames as well.

In [ ]:
def change_pivot(index_list):
    '''This function compares fall and spring scores within df_regression
    for the index values specified in index_list.'''
    df_change = df_regression.pivot_table(
    index = index_list, values = ['Fall', 'Spring'], 
                          aggfunc = 'mean').reset_index()
    df_change['Fall to Spring Change'] = (
    df_change['Spring'] - 
    df_change['Fall'])
    df_change.sort_values('Fall to Spring Change', ascending = False,
                         inplace = True)
    return df_change

In [ ]:
df_change_by_college = change_pivot(['gender'])
df_change_by_college

In [ ]:
df_change_by_college = change_pivot(['college'])
df_change_by_college

In [ ]:
df_change_by_level = change_pivot(
    index_list = ['level'])
df_change_by_level

In [ ]:
df_change_by_college_and_level = change_pivot(
    index_list = ['college', 'level'])
df_change_by_college_and_level

In [ ]:
px.bar(df_change_by_college_and_level, x = 'college',
       y = 'Fall to Spring Change', color = 'level',
      barmode = 'group',
      title = 'Average Fall-to-Spring Score Changes by College and Level')

### Creating a linear regression using a formula approach:

(The documentation at https://www.statsmodels.org/stable/examples/notebooks/generated/formulas.html proved very helpful in writing this section.)

In [ ]:
lr_model_1 = smf.ols(formula = "Spring ~ Fall + gender", 
                   data = df_regression) 
lr_results_1 = lr_model_1.fit()
lr_results_1.summary()

In [ ]:
lr_model_2 = smf.ols(formula = "Spring ~ Fall", 
                   data = df_regression) 
lr_results_2 = lr_model_2.fit()
lr_results_2.summary()

In [ ]:
lr_model_3 = smf.ols(formula = "Spring ~ Fall + level", 
                   data = df_regression) 
lr_results_3 = lr_model_3.fit()
lr_results_3.summary()

In [ ]:
lr_model_4 = smf.ols(formula = "Spring ~ Fall + college", 
                   data = df_regression) 
lr_results_4 = lr_model_4.fit()
lr_results_4.summary()

In [ ]:
lr_model_5 = smf.ols(formula = "Spring ~ Fall + college + level", 
                   data = df_regression) 
lr_results_5 = lr_model_5.fit()
params = lr_results_5.params # We'll use these in an upcoming demonstration
lr_results_5.summary()


In [ ]:
params

Predicting a spring survey score for a STM freshman with a fall score of 57:

In [ ]:
(57 * params['Fall'] 
 + params["college[T.STM]"] 
 + params['Intercept'])

In [ ]:
df_regression_pred_vs_actual = df_regression.copy().drop([
    'student_id', 'gender'], axis = 1)
df_regression_pred_vs_actual['Pred_Spring'] = lr_results_5.predict(
    df_regression_pred_vs_actual[['Fall', 'college', 'level']])
df_regression_pred_vs_actual['Actual-Pred'] = (
    df_regression_pred_vs_actual['Spring'] - 
    df_regression_pred_vs_actual['Pred_Spring'])
df_regression_pred_vs_actual

In [ ]:
lr_results_5.resid

In [ ]:
sms.omni_normtest(lr_results_1.resid)

In [ ]:
px.histogram(df_regression_pred_vs_actual, 
             x = 'Actual-Pred')

In [ ]:
df_regression_stm_fr = df_regression_pred_vs_actual.query(
    "college == 'STM' & level == 'Fr'").copy()
df_regression_stm_fr['Manual_Pred'] = (
    df_regression_stm_fr['Fall'] * params['Fall'] 
 + params["college[T.STM]"] 
 + params['Intercept'])
df_regression_stm_fr

In [ ]:
px.scatter(df_regression_pred_vs_actual, x = 'Spring', y = 'Actual-Pred')

In [ ]:
px.scatter(df_regression_pred_vs_actual, x = 'Pred_Spring', y = 'Spring')

Using the non-formula OLS approach:

(This code was based on the documentation found at https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.html .)

In [ ]:
df_regression['Score_Up'] = np.where(
    df_regression['Spring'] > df_regression['Fall'], 1, 0)
df_regression

In [ ]:
df_regression['Score_Up'].value_counts()

In [ ]:
lg_model_5 = smf.logit(
    formula = "Score_Up ~ Fall + college + level", data = df_regression) 
lg_results_5 = lg_model_5.fit()
params = lg_results_5.params # We'll use these in an upcoming demonstration
lg_results_5.summary()


## Importing data

In [ ]:
df_chvir = pd.read_csv(
    '../Census_Data_Imports/Datasets/home_val_income_data_county.csv')
# 'chvir' stands for 'state home values, incomes, and regions'--the 
# main data points provided for each county in the DataFrame.
# Simplifying our dataset by (1) condensing unwieldy column names and (2)
# removing extraneous fields:
df_chvir.rename(columns = {
    'Median Household Income in the Past 12 Months (in 2023 \
Inflation-Adjusted Dollars)_Estimate!!Median household income in the \
past 12 months (in 2023 inflation-adjusted dollars) \
(B19013_001E)':'Median_Income',
'Median Value (Dollars)_Estimate!!Median value \
(dollars) (B25077_001E)':'Median_Home_Value',
'Sex by Age_Estimate!!Total: (B01001_001E)':'Population'}, inplace = True)

df_chvir.drop(['Year', 'state', 'county'], axis = 1, inplace = True)
# Limiting our output to counties in one of the 50 US states (plus DC):
df_chvir.query("State_Abbrev != 'PR'", inplace = True)
df_chvir['Median_Income_Squared'] = df_chvir['Median_Income']**2
df_chvir

### Cleaning our dataset

Let's take a look at the average median incomes and home values within our dataset:

In [ ]:
df_chvir['Median_Income'].mean(), df_chvir['Median_Home_Value'].mean()

Well, these numbers don't seem right at all! What's going on with the data?

The issue here is that the number -666,666,666 is getting used as a code for missing home value and income statistics. The presence of this giant negative number within our dataset is dramatically skewing our average calculations.

I identified this number by calling .min() on the dataset's home value and income columns:

In [ ]:
df_chvir['Median_Home_Value'].min()

In [ ]:
df_chvir['Median_Income'].min()

I also looked for other unusually small values that might represent additional codes for invalid data, but I didn't find any:

In [ ]:
df_chvir.sort_values('Median_Home_Value')['Median_Home_Value'].unique()

In [ ]:
df_chvir.sort_values('Median_Income')['Median_Income'].unique()

I'll now remove these invalid entries from the dataset by filtering it to include only rows with *non-negative* median home values and median incomes. (I could have only filtered out rows with entries of -666666666, but that code would fail to work going forward if the Census Bureau happened to replace that number with a different one.)  

In [ ]:
df_chvir.query("Median_Home_Value >= 0 & Median_Income >= 0", inplace = True)
df_chvir.head()

In [ ]:
px.scatter(df_chvir, x = 'Median_Income', y = 'Median_Home_Value',
    color = 'Region', hover_data = 'NAME')

In [ ]:
df_chvir.query("Population > 50000", inplace = True)

In [ ]:
chvir_lr_1 = smf.ols(formula = "Median_Home_Value ~ Median_Income + \
Median_Income_Squared", 
                   data = df_chvir) 
chvir_lr_results_1 = chvir_lr_1.fit()
chvir_lr_results_1.summary()

In [ ]:
df_chvir_pred_vs_actual = df_chvir.copy().drop(
    ['Population', 'State_Abbrev'], axis = 1)
df_chvir_pred_vs_actual['Pred_Val'] = chvir_lr_results_1.predict(
    df_chvir_pred_vs_actual[['Median_Income', 'Median_Income_Squared']])
df_chvir_pred_vs_actual
df_chvir_pred_vs_actual['Actual-Pred'] = (
    df_chvir_pred_vs_actual['Median_Home_Value'] - 
    df_chvir_pred_vs_actual['Pred_Val'])
df_chvir_pred_vs_actual.sort_values('Actual-Pred')

In [ ]:
px.scatter(df_chvir_pred_vs_actual, x = 'Pred_Val', 
           y = 'Median_Home_Value', color = 'Region',
          hover_data = 'NAME')

In [ ]:
df_chvir_pred_vs_actual.sort_values('Actual-Pred').head(8)

In [ ]:
df_chvir_pred_vs_actual.sort_values(
    'Actual-Pred', ascending = False).head(8)

In [ ]:
px.histogram(x = chvir_lr_results_1.resid)

In [ ]:
chvir_lr_2 = smf.ols(formula = "Median_Home_Value ~ Median_Income + \
Median_Income_Squared + Region", 
                   data = df_chvir) 
chvir_lr_results_2 = chvir_lr_2.fit()
chvir_lr_results_2.summary()

In [ ]:
px.histogram(x = chvir_lr_results_2.resid)

In [ ]:
sms.omni_normtest(chvir_lr_results_2.resid)

## Performing state-level regressions:

In [ ]:
df_shvir = pd.read_csv(
    '../Census_Data_Imports/Datasets/home_val_income_data_state.csv')
# 'shvir' stands for 'state home values, incomes, and regions'--the 
# main data points provided for each county in the DataFrame.
# Simplifying our dataset by (1) condensing unwieldy column names and (2)
# removing extraneous fields:
df_shvir.rename(columns = {
    'Median Household Income in the Past 12 Months (in 2023 \
Inflation-Adjusted Dollars)_Estimate!!Median household income in the \
past 12 months (in 2023 inflation-adjusted dollars) \
(B19013_001E)':'Median_Income',
'Median Value (Dollars)_Estimate!!Median value \
(dollars) (B25077_001E)':'Median_Home_Value',
'Sex by Age_Estimate!!Total: (B01001_001E)':'Population'}, inplace = True)
df_shvir.query("State_Abbrev != 'PR'", inplace = True)
df_shvir.drop(['Year', 'state', 'State_Abbrev'], axis = 1, inplace = True)
# Limiting our output to counties in one of the 50 US states (plus DC):

df_shvir

In [ ]:
df_shvir['Median_Income_Squared'] = df_shvir['Median_Income']**2
df_shvir

## Performing a linear regression:

In [ ]:
shvir_lr_1 = smf.ols(
    formula = "Median_Home_Value ~ Median_Income + Median_Income_Squared", 
    data = df_shvir) 
shvir_lr_results_1 = shvir_lr_1.fit()
shvir_lr_results_1.summary()

In [ ]:
df_shvir_pred_vs_actual = df_shvir.copy().drop([
    'Population', 'Region'], axis = 1)
df_shvir_pred_vs_actual['Pred_Val'] = shvir_lr_results_1.predict(
    df_shvir_pred_vs_actual[['Median_Income', 'Median_Income_Squared']])
df_shvir_pred_vs_actual
df_shvir_pred_vs_actual['Actual-Pred'] = (
    df_shvir_pred_vs_actual['Median_Home_Value'] - 
    df_shvir_pred_vs_actual['Pred_Val'])
df_shvir_pred_vs_actual.sort_values('Actual-Pred').head(8)

In [ ]:
px.scatter(df_shvir_pred_vs_actual, x = 'Pred_Val', y = 'Median_Home_Value')

In [ ]:
df_shvir_pred_vs_actual.sort_values(
    'Actual-Pred', ascending = False).head(8)

In [ ]:
shvir_lr_2 = smf.ols(
    formula = "Median_Home_Value ~ Median_Income + Median_Income_Squared + Region", 
    data = df_shvir) 
shvir_lr_results_2 = shvir_lr_2.fit()
shvir_lr_results_2.summary()

In [ ]:
px.scatter(df_shvir, x = 'Median_Income', y = 'Median_Home_Value',
    color = 'Region', hover_data = 'NAME')

In [ ]:
sms.omni_normtest(shvir_lr_results_2.resid)